In [1]:
# -*- coding: UTF-8 -*-

from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd 
import numpy as np

## 第一步：获取两居的房源编号（以BJ开头) 
def get_lianjia_house_code():
    house_code_data=[]
    #print('程序开始时间{}'.format(time.time()))
    for i in range(1,101):
        time.sleep(5)
        url='https://bj.lianjia.com/zufang/pg'+str(i)+'rt200600000001l1/#contentList'
        #https://bj.lianjia.com/zufang/pg98rt200600000001l1l0/#contentList
        #print(url)
        ##https://bj.lianjia.com/zufang/rt200600000001l1l0/#contentList
        headers = {
        'Accept': 'application/json',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Cookie':  'lianjia_uuid=99f1e6f8-cc8d-435a-87f7-b0d360d6d5b2; _smt_uid=5cea4b11.2a13524e; _ga=GA1.2.1298220716.1558858518; _jzqa=1.4022354363956294700.1558858514.1559205609.1590936861.3; select_city=110000; _gid=GA1.2.1581606667.1610113282; csrfSecret=kWfghzqtT6sm4JCVrOoNXvhR; activity_ke_com=undefined; lianjia_ssid=be23dce7-ea64-4607-93d0-18e4ca13144c; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2216af3353f76675-0f15018c966e22-37677003-1296000-16af3353f7792b%22%2C%22%24device_id%22%3A%2216af3353f76675-0f15018c966e22-37677003-1296000-16af3353f7792b%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E4%BB%98%E8%B4%B9%E5%B9%BF%E5%91%8A%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%2C%22%24latest_utm_source%22%3A%22guanwang%22%2C%22%24latest_utm_medium%22%3A%22pinzhuan%22%2C%22%24latest_utm_campaign%22%3A%22wybeijing%22%2C%22%24latest_utm_content%22%3A%22biaotimiaoshu%22%2C%22%24latest_utm_term%22%3A%22biaoti%22%7D%7D; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1610113611; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1610113611; srcid=eyJ0Ijoie1wiZGF0YVwiOlwiMzQ3YjBkNjQzM2FlZDAyMTM2Yzg4Y2IwMTNhMjJkNGViODlhMGZhMGViYjc3NDdkZDdmYjFjOWNkN2E1MTk2ODg2ODYxOWM1MjA5MjhmNjU2MTgzMmFjZTRkZjNiYWU2M2E0YzBlZTVmOGQyNmQ3YzkxNzAwYTI3Y2E2MjZjNDU3YTUwMzZmMjAxNTBiYTU4M2QyMjA0ZWYxZjQ3NjY4NDEwZmY4ZmYyMDkxODJmMTljNmNmMGQzMjQ1NzYwZjZmYTkzOThiODI4NTdmZmQ1NmM4MzU5MTFmODUyN2JjY2Q3OGU5M2Q4YWFlNDk0OTk1YTdkYTk3ZDI3MmNjYTQzOTcyMTQ3ZDc1OGFmMGY5NmNlYjczZGFjMTBjZDdkY2Q2OGZjNjE0NjI4YmYxYmY2NWM1YWE2YTQyMmJjNWU4MmI3YWM3M2M3ZWYwMjc0MTVlZjAxNjNmZDQ1NTFkYWIyNFwiLFwia2V5X2lkXCI6XCIxXCIsXCJzaWduXCI6XCIyMDY3Zjk2ZlwifSIsInIiOiJodHRwczovL20ubGlhbmppYS5jb20vY2h1enUvYmovenVmYW5nL3J0MjAwNjAwMDAwMDAxbDEvIiwib3MiOiJ3ZWIiLCJ2IjoiMC4xIn0=',
        'Host': 'bj.lianjia.com',
        'Referer': 'https://bj.lianjia.com/zufang/pg1rt200600000001l1/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
        }
        
        html = requests.get(url, headers=headers).text
        bs = BeautifulSoup(html,'lxml')
        bs_house_code_list = bs.select('div[data-house_code]')
        ## 通过属性值获取房源编号,不使用正则，正则效率低
        ##house_code_re =re.compile(r' data-house_code="BJ\d*"')
        house_code_data=house_code_data+[i.attrs['data-house_code'] for i in bs_house_code_list]
    #print('程序结束时间{}'.format(time.time()))
    print(len(house_code_data))
    return house_code_data



## 第二步：获取每个房源的价格,朝向,面积，经纬度等信息，
def get_lianjia_house_detail(house_code_number):
    house_code_data=[]
    #print('程序开始时间{}'.format(time.time()))

    
    data_cc1={
        'house_code':[],
        'city_id':[],
        'house_condition_name':[],
        'beadroom_num':[],
        'rent_price':[],
        'name':[],
        'longitude':[],
        'latitude':[],
        'frame_hall_num':[],
        'subway':[]

    }
    
    data_cc2={
                'house_code':[],
                'area':[],
                'orientations':[],
                'storey':[],
                'elevator':[],
                'parking':[],
                'civil_water':[],
                'civil_power':[],
                'gas':[],
                'heating':[],
                'lease_term':[],

            }
    
    re_str=re.compile(r'g_conf.subway = (.*)') ## 20210201新增匹配地铁为止
    
    for i in range(len(house_code_number)):
        time.sleep(10)
        url='https://bj.lianjia.com/zufang/'+str(house_code_number[i])+'.html'+'?'+'nav=0&unique_id=99f1e6f8-cc8d-435a-87f7-b0d360d6d5b2zufangl1'+str(int(time.time()*1000))
        headers = {
        'Accept': 'application/json',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Cookie':  'lianjia_uuid=99f1e6f8-cc8d-435a-87f7-b0d360d6d5b2; _smt_uid=5cea4b11.2a13524e; _ga=GA1.2.1298220716.1558858518; _jzqa=1.4022354363956294700.1558858514.1559205609.1590936861.3; select_city=110000; _gid=GA1.2.1581606667.1610113282; csrfSecret=kWfghzqtT6sm4JCVrOoNXvhR; activity_ke_com=undefined; lianjia_ssid=be23dce7-ea64-4607-93d0-18e4ca13144c; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2216af3353f76675-0f15018c966e22-37677003-1296000-16af3353f7792b%22%2C%22%24device_id%22%3A%2216af3353f76675-0f15018c966e22-37677003-1296000-16af3353f7792b%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E4%BB%98%E8%B4%B9%E5%B9%BF%E5%91%8A%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%2C%22%24latest_utm_source%22%3A%22guanwang%22%2C%22%24latest_utm_medium%22%3A%22pinzhuan%22%2C%22%24latest_utm_campaign%22%3A%22wybeijing%22%2C%22%24latest_utm_content%22%3A%22biaotimiaoshu%22%2C%22%24latest_utm_term%22%3A%22biaoti%22%7D%7D; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1610113611; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1610113611; srcid=eyJ0Ijoie1wiZGF0YVwiOlwiMzQ3YjBkNjQzM2FlZDAyMTM2Yzg4Y2IwMTNhMjJkNGViODlhMGZhMGViYjc3NDdkZDdmYjFjOWNkN2E1MTk2ODg2ODYxOWM1MjA5MjhmNjU2MTgzMmFjZTRkZjNiYWU2M2E0YzBlZTVmOGQyNmQ3YzkxNzAwYTI3Y2E2MjZjNDU3YTUwMzZmMjAxNTBiYTU4M2QyMjA0ZWYxZjQ3NjY4NDEwZmY4ZmYyMDkxODJmMTljNmNmMGQzMjQ1NzYwZjZmYTkzOThiODI4NTdmZmQ1NmM4MzU5MTFmODUyN2JjY2Q3OGU5M2Q4YWFlNDk0OTk1YTdkYTk3ZDI3MmNjYTQzOTcyMTQ3ZDc1OGFmMGY5NmNlYjczZGFjMTBjZDdkY2Q2OGZjNjE0NjI4YmYxYmY2NWM1YWE2YTQyMmJjNWU4MmI3YWM3M2M3ZWYwMjc0MTVlZjAxNjNmZDQ1NTFkYWIyNFwiLFwia2V5X2lkXCI6XCIxXCIsXCJzaWduXCI6XCIyMDY3Zjk2ZlwifSIsInIiOiJodHRwczovL20ubGlhbmppYS5jb20vY2h1enUvYmovenVmYW5nL3J0MjAwNjAwMDAwMDAxbDEvIiwib3MiOiJ3ZWIiLCJ2IjoiMC4xIn0=',
        'Host': 'bj.lianjia.com',
        'Referer': 'https://bj.lianjia.com/zufang/pg1rt200600000001l1/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
        }
        html = requests.get(url, headers=headers).text
        bs = BeautifulSoup(html,'lxml')
        bs_0=bs.select('script')
        


        ## 获取第一部分数据
        map_columns_cc1={'g_conf.name':'name','g_conf.city_id':'city_id','g_conf.houseCode':'house_code','g_conf.houseConditionName':'house_condition_name'
                         ,'g_conf.bedroom_num':'beadroom_num','g_conf.rent_price':'rent_price'
                         ,'g_conf.frame_hall_num':'frame_hall_num'}



        for k in bs_0[5]:
            data_list=k.split('\n')
            
            for j in data_list:
                for cc1 in map_columns_cc1.items():
                    if j.replace(' ','').startswith(cc1[0]):
                        try:
                            data_cc1[cc1[1]].append(re.sub("(')|(=)|(;)|()|()|",'',j.replace(' ','').replace(cc1[0],'')))
                            #data_cc1[cc1[1]].append(j.replace(' ','').replace("'",'').replace('=','').replace(cc1[0],'').replace(';',''))
                        except:
                            data_cc1[cc1[1]].append(np.nan)
                            
                if j.replace(' ','').startswith('longitude'):
                    try:
                        data_cc1['longitude'].append(re.sub("(')|(=)|(longitude)|(;)|(:)|()|",'',j.replace(' ','')))
                    except:
                        data_cc1['longitude'].append(np.nan)
                elif j.replace(' ','').startswith('latitude'):
                    try:
                        data_cc1['latitude'].append(re.sub("(')|(=)|(latitude)|(;)|(:)|()|",'',j.replace(' ','')))
                    except:
                        data_cc1['latitude'].append(np.nan)
                elif j.replace(' ','').startswith('g_conf.subway'):
                    try:
                        data_cc1['subway'].append(re_str.search(j).group(1)) ## 使用正则
                    except:
                        data_cc1['subway'].append(np.nan)
                        



        ## 获取第二部分数据
        map_columns_cc2={'面积':'area','朝向':'orientations','楼层':'storey'
                     ,'电梯':'elevator','车位':'parking','用水':'civil_water','用电':'civil_power','燃气':'gas','采暖':'heating','租期':'lease_term'}


        data_cc2['house_code'].append(house_code_number[i])

        for cc2 in map_columns_cc2.items():
            try:
                data_cc2[cc2[1]].append(bs.find_all('li',class_='fl oneline',string=re.compile(cc2[0]))[0].getText().split('：')[1])
            except:
                data_cc2[cc2[1]].append(np.nan)
        if i%20==0:
            print(i) ##打印一些运行信息，目的为观察程序是运行的
    return data_cc1,data_cc2

            

if __name__ == '__main__':
    house_code_data_2 = get_lianjia_house_code()
    
    house_code_number=set(house_code_data_2)
    house_code_number_trim=[]
    for a in house_code_number:
        if a.startswith('BJ'):
            house_code_number_trim.append(a)
    
    data1,data2 = get_lianjia_house_detail(house_code_number_trim)
    
    ## 数据存储到本地，不需要索引值
    pd.DataFrame(data1).to_csv('北京两室租房房源信息_1.csv',index=False)
    pd.DataFrame(data2).to_csv('北京两室租房房源信息_2.csv',index=False)

3000
0
20
40
60
80
100
120
140
160


In [2]:
len(house_code_number)

165

In [69]:
data1['subway'][4]

'[{"distance":887,"lines":["\\u516b\\u901a\\u7ebf"],"name":"\\u9ad8\\u7891\\u5e97","point_lat":39.916198,"point_lng":116.537052},{"distance":1039,"lines":["1\\u53f7\\u7ebf","\\u516b\\u901a\\u7ebf"],"name":"\\u56db\\u60e0\\u4e1c","point_lat":39.914741,"point_lng":116.52183},{"distance":1103,"lines":["6\\u53f7\\u7ebf"],"name":"\\u9752\\u5e74\\u8def","point_lat":39.929697,"point_lng":116.525295}];'

In [8]:
'\\u56db\\u60e0'.encode('utf-8').decode('unicode_escape')

'四惠'

In [72]:
data1['house_code'][4]

'BJ2710005715749044224'

In [70]:
import json
str_cc=data1['subway'][4]
re_str=re.compile(r'\{(.*)\}')

In [73]:
str_tt=re_str.search(str_cc).group(0)
print(json.loads(str_tt.replace('},{','};{').split(';')[0]))
print(json.loads(str_tt.replace('},{','};{').split(';')[1]))
print(json.loads(str_tt.replace('},{','};{').split(';')[2]))

{'distance': 887, 'lines': ['八通线'], 'name': '高碑店', 'point_lat': 39.916198, 'point_lng': 116.537052}
{'distance': 1039, 'lines': ['1号线', '八通线'], 'name': '四惠东', 'point_lat': 39.914741, 'point_lng': 116.52183}
{'distance': 1103, 'lines': ['6号线'], 'name': '青年路', 'point_lat': 39.929697, 'point_lng': 116.525295}


In [40]:
str_tt2={"distance":462,"lines":["1\u53f7\u7ebf","\u516b\u901a\u7ebf"],"name":"\u56db\u60e0","point_lat":39.913735,"point_lng":116.502239},{"distance":1019,"lines":["1\u53f7\u7ebf","\u516b\u901a\u7ebf"],"name":"\u56db\u60e0\u4e1c","point_lat":39.915068,"point_lng":116.52184}

type(str_tt2)


tuple

In [47]:
cc='aaaabbbbddd'
cc.split('bb')

['aaaa', '', 'ddd']

In [54]:

import json
 
str = '''
[{
    "name": "Tom",
    "gender": "male"
}, {
    "name": "Jack",
    "gender": "male"   
}]
'''
#将字符串转为json格式
print(type(str))
data = json.loads(str)
print(type(data))
print(data)

<class 'str'>
<class 'list'>
[{'name': 'Tom', 'gender': 'male'}, {'name': 'Jack', 'gender': 'male'}]


In [55]:
print(type(data[0]))

<class 'dict'>


In [59]:
dd=[1,2,3]
str(dd)

TypeError: 'str' object is not callable

In [77]:
from scipy import integrate
import numpy as np
def f(x):
    return 0.6621*(np.power(x,-0.542)) 
v, _ = integrate.quad(f, 0,365) ## 计算用户一年的LT
print(v)

21.556996032781562


In [75]:
print(365*2)

730
